In [ ]:
## The code in this notebook ist in part taken from this repository.
## Credit to: F.  Radenovic,  A.  Iscen,  G.  Tolias,  Y.  Avrithis,  and  O.  Chum.  Revisitingoxford  and  paris:   Large-scale  image  retrieval  benchmarking.   InCVPR,2018.
!rm -rf revisitop
!git clone https://github.com/filipradenovic/revisitop.git
data_root = 'revisitop/data/'
from revisitop.python.download import download_datasets
download_datasets(data_root)

Cloning into 'revisitop'...
remote: Enumerating objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 34.17 KiB | 6.83 MiB/s, done.
Resolving deltas: 100% (80/80), done.
>> Dataset roxford5k directory does not exist. Creating: revisitop/data/datasets/roxford5k/jpg
>> Downloading dataset roxford5k archive oxbuild_images.tgz...
>> Extracting dataset roxford5k archive oxbuild_images.tgz...
>> Extracted, deleting dataset roxford5k archive oxbuild_images.tgz...
>> Downloading dataset roxford5k ground truth file...
>> Dataset rparis6k directory does not exist. Creating: revisitop/data/datasets/rparis6k/jpg
>> Downloading dataset rparis6k archive paris_1.tgz...
>> Extracting dataset rparis6k archive paris_1.tgz...
>> Extracted, deleting dataset rparis6k archive paris_1.tgz...
>> Downloading dataset rparis6k archive paris_2.tgz...
>> Extracting dataset rparis6k archive paris_2.tgz...
>> Extracted, deleting dataset rparis6k archi

In [ ]:
!rm -rf functions
!git clone https://github.com/Lennard-Alms/lab_bd.git functions

Cloning into 'functions'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 974 (delta 138), reused 164 (delta 73), pack-reused 743
Receiving objects: 100% (974/974), 692.59 KiB | 15.74 MiB/s, done.
Resolving deltas: 100% (597/597), done.


In [ ]:
import os
import numpy as np
from PIL import Image, ImageFile
from functions.preprocessing.BuildDB import extract_features
from functions.model.CNN_Strats import build_model
from functions.preprocessing.HelperFunctions import get_image, pil_loader
from revisitop.python.evaluate import compute_map
from revisitop.python.dataset import configdataset
import matplotlib.pyplot as plt
import numpy as np
import gc
import cv2
import tensorflow as tf

In [ ]:
data_root = 'revisitop/data/'
test_dataset = 'roxford5k'
cfg = configdataset(test_dataset, os.path.join(data_root, 'datasets'))
gnd = cfg['gnd']
ks = [1, 5, 10]

In [ ]:
max_shape = (1024, 1024, 3)
EXP = [3,5]
DEP = [5]
with open('output.txt', 'w') as f:
  f.write('VGG REGIONS \n')

In [ ]:
for exp in EXP:
  for dep in DEP: 
    imodel = build_model(max_shape, exp, vgg_output=True, regions=True, depth=dep)
    im_paths = np.array([cfg['im_fname'](cfg, i) for i in np.arange(cfg['n'])])
    im_features = extract_features(im_paths, imodel, batch_size=20, network_batch_size=2, pil=True, shape=max_shape, fill_option='white', concat=False)
    q_features = []
    for i in np.arange(cfg['nq']):
      qim = np.array(pil_loader(cfg['qim_fname'](cfg, i)).crop(gnd[i]['bbx']))
      qmodel = build_model(qim.shape, exp, vgg_output=True, mac=True)
      q_features.append(extract_features([qim], qmodel, batch_size=100, network_batch_size=4))
    q_features = np.concatenate(q_features)

    im_features_all = []
    for batch in range(len(im_features)):
      for im in range(len(im_features[batch][0])):
        all_im = []
        for size in range(len(im_features[0])):
          for y in range(len(im_features[batch][size][im])):
            for x in range(len(im_features[batch][size][im][y])):
              all_im.append(im_features[batch][size][im][y][x])
        im_features_all.append(all_im)
    im_features_all = np.array(im_features_all)
    final_dists = []
    for im_feat in im_features_all:
      dist = np.arccos(np.dot(im_feat, q_features.T))
      sdist = np.sort(dist, axis=0)
      final_dists.append((sdist[0]))
    final_dists = np.array(final_dists)
    ranks = np.argsort(final_dists, axis=0)

    # search for easy
    gnd_t = []
    for i in range(len(gnd)):
        g = {}
        g['ok'] = np.concatenate([gnd[i]['easy']])
        g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
        gnd_t.append(g)
    mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t, ks)

    # search for easy & hard
    gnd_t = []
    for i in range(len(gnd)):
        g = {}
        g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
        g['junk'] = np.concatenate([gnd[i]['junk']])
        gnd_t.append(g)
    mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t, ks)

    # search for hard
    gnd_t = []
    for i in range(len(gnd)):
        g = {}
        g['ok'] = np.concatenate([gnd[i]['hard']])
        g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
        gnd_t.append(g)
    mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, ks)
    with open('output.txt', 'a') as f:
      f.write('=========================================== \n')
      f.write('vgg rmac exp:' + str(exp) + ' dep:' + str(dep) + ' \n')
      f.write('>> {}: mAP E: {}, M: {}, H: {} \n'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
      f.write('>> {}: mP@k{} E: {}, M: {}, H: {} \n'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))
    print('>> {}: mAP E: {}, M: {}, H: {}'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
    print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))

1/1 [==============================] - 0s 196ms/step
>> roxford5k: mAP E: 67.23, M: 48.44, H: 20.53
>> roxford5k: mP@k[ 1  5 10] E: [91.18 83.68 75.83], M: [88.57 78.57 70.43], H: [50.   40.57 33.43]
1/1 [==============================] - 0s 247ms/step
>> roxford5k: mAP E: 66.31, M: 47.4, H: 19.82
>> roxford5k: mP@k[ 1  5 10] E: [92.65 82.65 74.5 ], M: [90.   77.43 67.86], H: [48.57 39.43 32.86]
